### Security Hub
##### Minimal Cost Configuration

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account)
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ### STANDARDS ###
            securityhub = session.client('securityhub', region_name = region['RegionName'])
            paginator = securityhub.get_paginator('get_enabled_standards')
            pages = paginator.paginate()
            for page in pages:
                for subscriptions in page['StandardsSubscriptions']:
                    securityhub.batch_disable_standards(
                        StandardsSubscriptionArns = [
                            subscriptions['StandardsSubscriptionArn']
                        ]
                    )
            ### SUBSCRIPTIONS ###
            paginator = securityhub.get_paginator('list_enabled_products_for_import')
            pages = paginator.paginate()
            for page in pages:
                for products in page['ProductSubscriptions']:
                    parsed = products.split('/')
                    if parsed[2] != 'access-analyzer' and parsed[2] != 'health' and parsed[2] != 'securityhub':
                        securityhub.disable_import_findings_for_product(
                            ProductSubscriptionArn = products
                        )
            print(' - '+region['RegionName'])
        except:
            print(' - '+region['RegionName']+' DENIED')
            pass